# Exercise 6

# Setup to run the notebook

- Ensure that `python3` is available in the system with python version 3.10.
- Create a virtual env - `python3 -m venv env`
- Activate the env - `source env/bin/activate`
- Run the code cells for observing results.

## 1: The ElGamal cryptosystem

In [4]:
import random

In [5]:
# Helper function
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

# Helper function
def find_primitive_root(q):
    for g in range(2, q):
        if gcd(g, q) == 1:
            return g
    return None

# Key generation for the ElGamal cryptosystem
def e_keygen(q):
    g = find_primitive_root(q)
    if g is None:
        return None, None
    x = random.randint(1, q-2)
    h = pow(g, x, q)

    public_key = (q, g, h)
    private_key = x

    return public_key, private_key

In [6]:
# Key encryption for the ElGamal cryptosystem
def e_encrypt(msg, pk):
    q, g, h = pk
    k = random.randint(1, q-2)

    c1 = pow(g, k, q)
    c2 = (msg * pow(h, k, q)) % q

    return c1, c2

In [7]:
# Helper function
def modinv(a, m):
    # This function returns the inverse of a modulo m
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('Modular inverse does not exist')
    else:
        return x % m

# Helper function
def egcd(a, b):
    # Extended Euclidean Algorithm
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

# Key decryption for the ElGamal cryptosystem
def e_decrypt(c, x, pk):
    c1, c2 = c
    q, g, h = pk

    s = pow(c1, x, q)
    s_inv = modinv(s, q)
    msg = (c2 * s_inv) % q
    return msg

In [8]:
# Test the ElGamal cryptosystem for generating keys, encrypting and decrypting
q = 2**127 - 1

sk, pk = e_keygen(q)
c = e_encrypt(50, sk)

e_decrypt(c, pk, sk)

50

In [9]:
# Multiplication of ElGamal ciphertexts
def e_mult(c1, c2, pk):
    q, g, h = pk
    c11, c12 = c1
    c21, c22 = c2

    # Multiply the first parts of the ciphertexts
    new_c1 = (c11 * c21) % q

    # Multiply the second parts of the ciphertexts
    new_c2 = (c12 * c22) % q

    return new_c1, new_c2

In [10]:
# Test cases for showing correctness of the implementation

# Test 1
q = 2**127 - 1
pk, sk = e_keygen(q)
c1 = e_encrypt(5, pk)
c2 = e_encrypt(10, pk)
c3 = e_mult(c1, c2, pk)

assert e_decrypt(c3, sk, pk) == 50

# Test 2
q = 2**227 - 1
pk, sk = e_keygen(q)
c1 = e_encrypt(5**25, pk)
c2 = e_encrypt(10**25, pk)
c3 = e_mult(c1, c2, pk)

assert e_decrypt(c3, sk, pk) == 2980232238769531250000000000000000000000000


# Test 3
q = 2**327 - 1
pk, sk = e_keygen(q)
c1 = e_encrypt(28**50, pk)
c2 = e_encrypt(329**50, pk)
c3 = e_mult(c1, c2, pk)

assert e_decrypt(c3, sk, pk) == 227649946206217048085015677443757528434845177408940388249825297116688924748709775450598383345711419

## 2: Application of partially homomorphic encryption: Paillier

Q: Which operations does the Pailler encryption scheme support?

A: It supports the following operations
- Homomorphic Addition of Ciphertexts
- Homomorphic Multiplication of an Encrypted Number by a Plaintext Number


Q: Which data types does it operate on?

A: The Paillier cryptosystem operates on integers.



In [9]:
# Install dependencies

%pip install numpy
%pip install phe
%pip install concrete-numpy


In [1]:
import numpy as np
import concrete.numpy as cnp
import phe as paillier

In [2]:
class Client:
    def __init__(self):
        self.public_key, self.private_key = paillier.generate_paillier_keypair()

    def encrypt_cart(self, cart):
        # Encrypts only quantities in the cart
        encrypted_cart = [(price, self.public_key.encrypt(quantity)) for price, quantity in cart]
        return encrypted_cart

    def decrypt_total(self, encrypted_total):
        # Decrypts the total cost
        return self.private_key.decrypt(encrypted_total)


class Server:
    def compute_encrypted_total(self, encrypted_cart):
        # Start with an encrypted zero
        total_encrypted = 0
        for price, encrypted_quantity in encrypted_cart:
            # Add each price multiplied by its encrypted quantity to the total
            total_encrypted += price * encrypted_quantity
        return total_encrypted

# Example usage
cart = [
    # (price, quantity)
    (2000, 1),
    (120, 5),
    (1999, 3),
]

client = Client()
encrypted_cart = client.encrypt_cart(cart)
server = Server()
encrypted_total = server.compute_encrypted_total(encrypted_cart)
total_cost = client.decrypt_total(encrypted_total)

expected_price = 8597
assert total_cost == expected_price
print("Total cost:", total_cost)


Total cost: 8597


## 3: Application of partially homomorphic encryption: ElGamal



In [ ]:
class Client_EG:
    def client(self, cart):
        q = 2**127 - 1
        pk, sk = e_keygen(q)
        total_en = 0

        for price, quantity in cart:
            en_price = e_encrypt(price, pk)
            en_quantity = e_encrypt(quantity, pk)
            total_en += e_decrypt(Server_EG.price_calculator(en_price, en_quantity, pk),sk ,pk)
        return total_en

class Server_EG:
    def price_calculator(en_price, en_quantity, pk):
      en_total = e_mult(en_price, en_quantity, pk)
      return en_total

# Example cart with (price, quantity) pairs
cart = [
    (2000, 1),
    (120, 5),
    (1999, 3),
]


client = Client_EG()
total_cost = client.client(cart)

# Verify the result
expected_price = 8597
assert total_cost == expected_price
print("Total cost:", total_cost)

Total cost: 8597


Q: Which changes did you have to make?

A:

- For ElGamal the code had to be changed so that the additions do not happen in the calculator. Instead, the quantity is encrypted and multiplied by the encrypted price.

- With Paillier, the calculator sees the price in plain text. The rest is encrypted. For ElGamal, everything is encrypted for the calculator.


Q: What do your changes mean for the client (which advantages and/o disadvantages does the client have)?

A:

- The server can only multiply encrypted values, not add them. This means the server can't directly calculate the total price if both price and quantity are encrypted.

- The client eventually has to decrypt the final aggregated result. Therefore we have more privacy for the client with the elGamal calculator but there the client has to add the sums itself.

- In EIGamal cryptosystem, we can perform the multiplication of two encrypted values. We cannot directly add two encrypted values, which is a key operation needed for calculating the total price in the shopping cart.

## 4: Fully homomorphic encryption


In [12]:
import concrete.numpy as cnp

@cnp.compiler({"x": "encrypted"})
def add(x):
    return np.sum(x) // 6

example = [[1, 2, 4, 5, 7, 4]]
circuit = add.compile(example)
result = circuit.encrypt_run_decrypt(*example)
clear_evaluation = np.mean(example)

print(f"Evaluation of mean (plain) = {clear_evaluation}, homomorphically = {result}")

Evaluation of mean (plain) = 3.8333333333333335, homomorphically = 3


We cannot have floating point output in concrete numpy - https://docs.zama.ai/concrete/2.6/core-features/floating_points
> Concrete partly supports floating points. There is no support for floating point inputs or outputs. However, there is support for intermediate values to be floating points (under certain constraints).

We use Pyfhel that have floating point support.




In [30]:
%pip install Pyfhel

  Using cached Pyfhel-3.4.2.tar.gz (1.0 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for Pyfhel: filename=Pyfhel-3.4.2-cp310-cp310-linux_x86_64.whl size=9679269 sha256=f7f3979e879e3de454c31e3f8ea1d973e51df1cd654642c63b6f87c68b837aaa
  Stored in directory: /root/.cache/pip/wheels/74/7b/34/876b6d79c02f65e68e4e6f262735c2347cf77581ac50ca470d
Successfully built Pyfhel


In [40]:
from Pyfhel import Pyfhel

def homomorphic_mean_2(HE, x):
    # Start with an encrypted zero
    encrypted_sum = HE.encrypt(0)
    for value in x:
        temp_ctxt = HE.encrypt(value)
        encrypted_sum += temp_ctxt

    # Decrypt the sum
    decrypted_sum = HE.decrypt(encrypted_sum, decode=False)
    integer_sum = HE.decode(decrypted_sum)

    # Calculate the mean
    mean = np.around(integer_sum / len(x), 2)
    return mean[0]

def generate_ckks_context():
    HE = Pyfhel()
    try:
        HE.contextGen(scheme='BFV', n=4096, t_bits=20, sec=128)
        HE.keyGen()
    except Exception as e:
        print(f"Error initializing Pyfhel: {e}")
        raise

    return HE


example = [1, 2, 4, 5, 7, 4]
HE2 = generate_ckks_context()
homomorphic_evaluation = homomorphic_mean_2(HE2, example)
clear_evaluation = np.mean(example)
print(f"Evaluation of mean (plain) = {clear_evaluation}, homomorphically = {homomorphic_evaluation}")

Evaluation of mean (plain) = 3.8333333333333335, homomorphically = 3.83


Q: How can you obtain the mean of a list with 7 encrypted integers?

A: To compute the mean of a list of 7 encrypted integers, We would modify the mean to divide by 7 instead of 6.

Q: What are the limitations of your implementations?

A: Limitations are as follows:

- Precision: The implementation provides only integer results due to the limitations of homomorphic encryption in handling floating-point operations.

- Performance and Complexity: Homomorphic encryption is computationally expensive and not suitable for very large datasets or highly complex operations.

- Encrypted Data Size: The size of the encrypted data is significantly larger than the plaintext, which may lead to increased storage and transmission costs.